In [2]:
import numpy as np
import re
import time
import pandas as pd

# spatial imports
import geopandas as gpd
import pygeos

# FCA specific imports
from access import Access, weights

/Users/hanna/opt/anaconda3/lib/python3.8/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.1-CAPI-1.14.2). Conversions between both will be slow.
  warnings.warn(


In [3]:
supply = pd.read_csv("/college_data/two_year_tr_supply.csv") # two_year_ct_supply.csv ... change inputs here
demand = pd.read_csv("/college_data/two_year_tr_demand.csv")
times = pd.read_csv("/college_data/two_year_tr_drive_times.csv")
times = times.dropna()
demand = demand.dropna()
supply = supply.dropna()


FileNotFoundError: [Errno 2] No such file or directory: '/college_data/two_year_tr_supply.csv'

In [ ]:
#demand

In [22]:
# load in times matrix - converting columns to integers
times.origin = times.origin.astype(int)
times.dest = times.dest.astype(int)

# load in supply matrix - converting columns to integers
supply.GEOID = supply.GEOID.astype(int)

# load in demand matrix - converting columns to integers
demand.GEOID = demand.GEOID.astype(int)


In [23]:
# weight for enchanced fca
fn30 = weights.step_fn({10:1, 20:0.68, 30:0.22})
fn30

<function access.weights.step_fn.<locals>.helper(key_to_test)>

In [24]:
# create Access object
A = Access(demand_df = demand,
           demand_index = "GEOID",
           #demand_index = "health_district",
           demand_value = "popE",
           supply_df = supply,
           supply_index = "GEOID",
           #supply_index = "health_district",
           supply_value = "capacity",
           cost_df = times,
           cost_origin = "origin",
           cost_dest = "dest",
           cost_name = "cost",
          neighbor_cost_df = times,
           neighbor_cost_origin = "origin",
           neighbor_cost_dest = "dest",
           neighbor_cost_name = "cost")
A

In [25]:
# obtain fcas
info_fca5 = A.fca_ratio(name = "fca5", max_cost = 5)
info_fca10 = A.fca_ratio(name = "fca10", max_cost = 10)
info_fca15 = A.fca_ratio(name = "fca15", max_cost = 15)
info_2sfca5 = A.two_stage_fca(name = "2sfca5", max_cost = 5)
info_2sfca10 = A.two_stage_fca(name = "2sfca10", max_cost = 10)
info_2sfca15 = A.two_stage_fca(name = "2sfca15", max_cost = 15)


/Users/hanna/opt/anaconda3/lib/python3.8/site-packages/access/fca.py:137: UserWarning: some tracts may be unaccounted for in supply_cost
  warnings.warn("some tracts may be unaccounted for in supply_cost", stacklevel = 1)
/Users/hanna/opt/anaconda3/lib/python3.8/site-packages/access/fca.py:137: UserWarning: some tracts may be unaccounted for in supply_cost
  warnings.warn("some tracts may be unaccounted for in supply_cost", stacklevel = 1)
/Users/hanna/opt/anaconda3/lib/python3.8/site-packages/access/fca.py:137: UserWarning: some tracts may be unaccounted for in supply_cost
  warnings.warn("some tracts may be unaccounted for in supply_cost", stacklevel = 1)


In [26]:
enh_2sfca = A.enhanced_two_stage_fca(name = "2sfca30", weight_fn = fn30)
info_3sfca = A.three_stage_fca(name = "3sfca", normalize=True)
df = pd.concat([info_3sfca, enh_2sfca], axis=1)
df.describe()

,3sfca_capacity,2sfca30_capacity
count,1899.000000,1899.000000
mean,1.105710,0.000578
std,1.711374,0.001002
min,0.000000,0.000000
25%,0.116916,0.000000
50%,0.648055,0.000260
75%,1.309708,0.000675
max,14.410786,0.008246


In [27]:
df.to_csv("/college_data/3sfca_two_year_tr.csv")